In [1]:
! pip3 install transformers=='4.48.3'


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
! pip install beir=='2.0.0'


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
! pip3 install InstructorEmbedding


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
! pip3 install sentence-transformers


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [5]:
! nvidia-smi

Wed Mar 19 10:46:46 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.77                 Driver Version: 565.77         CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA RTX A5000               On  |   00000000:03:00.0 Off |                  Off |
|  0%   35C    P8             28W /  230W |       2MiB /  24564MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
import os
import pathlib
import logging
from datetime import timedelta
from typing import List, Dict, Union, Tuple

import numpy as np
import torch
from torch import Tensor
import torch.distributed as dist
from tqdm import trange
from transformers import AutoTokenizer, AutoModel
from transformers.file_utils import PaddingStrategy

from beir import util, LoggingHandler
from beir.retrieval import models
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES

import torch
import numpy as np
from typing import List, Dict
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModel
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from sentence_transformers import SentenceTransformer

# Configure logging
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)


class InstructorDenseRetriever:
    def __init__(self, model_path="hkunlp/instructor-large"):
        """Initialize the Instructor Embedding model for both query and document encoding"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # Load Model
        self.model = SentenceTransformer(model_path, device=self.device)

    def encode_queries(self, queries: List[str], batch_size: int = 64, **kwargs) -> np.ndarray:
        """Encodes queries into dense embeddings using instructional prompts"""
        instruction = "Represent the query for retrieving supporting documents:"
        query_embeddings = []

        with torch.no_grad():
            for start_idx in trange(0, len(queries), batch_size):
                batch_queries = queries[start_idx:start_idx+batch_size]
                embeddings = self.model.encode(batch_queries, batch_size=batch_size, convert_to_numpy=True, prompt=instruction)
                query_embeddings.append(embeddings)

        return np.vstack(query_embeddings)

    def encode_corpus(self, corpus: List[Dict[str, str]], batch_size: int = 64, **kwargs) -> np.ndarray:
        """Encodes documents into dense embeddings using instructional prompts"""
        instruction = "Represent the document for retrieval:"
        corpus_embeddings = []

        with torch.no_grad():
            for start_idx in trange(0, len(corpus), batch_size):
                batch_corpus = [row.get('title', '') + " " + row.get('text', '') for row in corpus[start_idx:start_idx+batch_size]]
                embeddings = self.model.encode(batch_corpus, batch_size=batch_size, convert_to_numpy=True, prompt=instruction)
                corpus_embeddings.append(embeddings)

        return np.vstack(corpus_embeddings)

In [7]:
instructor = DRES(InstructorDenseRetriever())
retriever_instructor = EvaluateRetrieval(instructor, score_function="cos_sim") # or "cos_sim" for cosine similarity

2025-03-19 10:47:18 - Load pretrained SentenceTransformer: hkunlp/instructor-large


In [8]:
# Download and load dataset
dataset = "nfcorpus" # dataset name
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
out_dir = "datasets"
data_path = util.download_and_unzip(url, out_dir)

In [9]:
# Load corpus, queries, and qrels
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2025-03-19 10:47:25 - Loading Corpus...


  0%|          | 0/3633 [00:00<?, ?it/s]

2025-03-19 10:47:25 - Loaded 3633 TEST Documents.
2025-03-19 10:47:25 - Doc Example: {'text': 'Recent studies have suggested that statins, an established drug group in the prevention of cardiovascular mortality, could delay or prevent breast cancer recurrence but the effect on disease-specific mortality remains unclear. We evaluated risk of breast cancer death among statin users in a population-based cohort of breast cancer patients. The study cohort included all newly diagnosed breast cancer patients in Finland during 1995–2003 (31,236 cases), identified from the Finnish Cancer Registry. Information on statin use before and after the diagnosis was obtained from a national prescription database. We used the Cox proportional hazards regression method to estimate mortality among statin users with statin use as time-dependent variable. A total of 4,151 participants had used statins. During the median follow-up of 3.25 years after the diagnosis (range 0.08–9.0 years) 6,011 participants die

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

In [10]:
import json
# load paraphrased dataset
with open(f"{dataset}_query_paraphrased_gpt4o.json", encoding='utf-8') as f:
    # Load the JSON data into a Python dictionary
    queries_para = json.load(f)

In [11]:
queries_p = {}
for q in queries_para:
  queries_p[q] = queries_para[q]['query_p']

In [12]:
results_instructor = retriever_instructor.retrieve(corpus, queries_p)

2025-03-19 10:47:39 - Encoding Queries...


  0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:01,  1.55it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

 93%|█████████▎| 27/29 [01:47<00:05,  2.67s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 97%|█████████▋| 28/29 [01:48<00:02,  2.30s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:49<00:00,  3.77s/it]


In [13]:
# Evaluate your model with NDCG@k, MAP@K, Recall@K and Precision@K where k = [1,3,5,10,100,1000]
ndcg, _map, recall, precision = retriever_instructor.evaluate(qrels, results_instructor, retriever_instructor.k_values)

2025-03-19 10:50:43 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2025-03-19 10:50:43 - 

2025-03-19 10:50:43 - NDCG@1: 0.3669
2025-03-19 10:50:43 - NDCG@3: 0.3288
2025-03-19 10:50:43 - NDCG@5: 0.3194
2025-03-19 10:50:43 - NDCG@10: 0.2905
2025-03-19 10:50:43 - NDCG@100: 0.2818
2025-03-19 10:50:43 - NDCG@1000: 0.3739
2025-03-19 10:50:43 - 

2025-03-19 10:50:43 - MAP@1: 0.0411
2025-03-19 10:50:43 - MAP@3: 0.0668
2025-03-19 10:50:43 - MAP@5: 0.0808
2025-03-19 10:50:43 - MAP@10: 0.0956
2025-03-19 10:50:43 - MAP@100: 0.1287
2025-03-19 10:50:43 - MAP@1000: 0.1438
2025-03-19 10:50:43 - 

2025-03-19 10:50:43 - Recall@1: 0.0411
2025-03-19 10:50:43 - Recall@3: 0.0789
2025-03-19 10:50:43 - Recall@5: 0.1021
2025-03-19 10:50:43 - Recall@10: 0.1386
2025-03-19 10:50:43 - Recall@100: 0.3077
2025-03-19 10:50:43 - Recall@1000: 0.6407
2025-03-19 10:50:43 - 

2025-03-19 10:50:43 - P@1: 0.3839
2025-03-19 10:50:43

In [14]:
print(f"Model: Instructor; Dataset: {dataset} (paraphrased)")
print("-" * 150)
print(ndcg)
print(_map)
print(recall)
print(precision)

Model: Instructor; Dataset: nfcorpus (paraphrased)
------------------------------------------------------------------------------------------------------------------------------------------------------
{'NDCG@1': 0.36687, 'NDCG@3': 0.32884, 'NDCG@5': 0.3194, 'NDCG@10': 0.29052, 'NDCG@100': 0.28182, 'NDCG@1000': 0.37394}
{'MAP@1': 0.04105, 'MAP@3': 0.06679, 'MAP@5': 0.08082, 'MAP@10': 0.09556, 'MAP@100': 0.1287, 'MAP@1000': 0.14384}
{'Recall@1': 0.04105, 'Recall@3': 0.07888, 'Recall@5': 0.10214, 'Recall@10': 0.13861, 'Recall@100': 0.30772, 'Recall@1000': 0.64068}
{'P@1': 0.3839, 'P@3': 0.31579, 'P@5': 0.2904, 'P@10': 0.22353, 'P@100': 0.07898, 'P@1000': 0.02117}


In [15]:
# Download and load dataset
dataset = "scifact" # dataset name
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
out_dir = "datasets"
data_path = util.download_and_unzip(url, out_dir)

In [16]:
# Load corpus, queries, and qrels
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2025-03-19 10:50:53 - Loading Corpus...


  0%|          | 0/5183 [00:00<?, ?it/s]

2025-03-19 10:50:53 - Loaded 5183 TEST Documents.
2025-03-19 10:50:53 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

In [17]:
import json
# load paraphrased dataset
with open(f"{dataset}_query_paraphrased_gpt4o.json", encoding='utf-8') as f:
    # Load the JSON data into a Python dictionary
    queries_para = json.load(f)

In [18]:
queries_p = {}
for q in queries_para:
  queries_p[q] = queries_para[q]['query_p']

In [19]:
results_instructor = retriever_instructor.retrieve(corpus, queries_p)

2025-03-19 10:50:59 - Encoding Queries...


  0%|          | 0/3 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 33%|███▎      | 1/3 [00:00<00:01,  1.79it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

 95%|█████████▌| 39/41 [02:26<00:04,  2.27s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 98%|█████████▊| 40/41 [02:28<00:02,  2.09s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 41/41 [02:28<00:00,  3.63s/it]


In [20]:
# Evaluate your model with NDCG@k, MAP@K, Recall@K and Precision@K where k = [1,3,5,10,100,1000]
ndcg, _map, recall, precision = retriever_instructor.evaluate(qrels, results_instructor, retriever_instructor.k_values)

2025-03-19 10:53:29 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2025-03-19 10:53:29 - 

2025-03-19 10:53:29 - NDCG@1: 0.5133
2025-03-19 10:53:29 - NDCG@3: 0.5708
2025-03-19 10:53:29 - NDCG@5: 0.6012
2025-03-19 10:53:29 - NDCG@10: 0.6226
2025-03-19 10:53:29 - NDCG@100: 0.6512
2025-03-19 10:53:29 - NDCG@1000: 0.6655
2025-03-19 10:53:29 - 

2025-03-19 10:53:29 - MAP@1: 0.4892
2025-03-19 10:53:29 - MAP@3: 0.5454
2025-03-19 10:53:29 - MAP@5: 0.5654
2025-03-19 10:53:29 - MAP@10: 0.5756
2025-03-19 10:53:29 - MAP@100: 0.5828
2025-03-19 10:53:29 - MAP@1000: 0.5833
2025-03-19 10:53:29 - 

2025-03-19 10:53:29 - Recall@1: 0.4892
2025-03-19 10:53:29 - Recall@3: 0.6151
2025-03-19 10:53:29 - Recall@5: 0.6916
2025-03-19 10:53:29 - Recall@10: 0.7518
2025-03-19 10:53:29 - Recall@100: 0.8750
2025-03-19 10:53:29 - Recall@1000: 0.9867
2025-03-19 10:53:29 - 

2025-03-19 10:53:29 - P@1: 0.5133
2025-03-19 10:53:29

In [21]:
print(f"Model: Instructor; Dataset: {dataset} (paraphrased)")
print("-" * 150)
print(ndcg)
print(_map)
print(recall)
print(precision)

Model: Instructor; Dataset: scifact (paraphrased)
------------------------------------------------------------------------------------------------------------------------------------------------------
{'NDCG@1': 0.51333, 'NDCG@3': 0.57079, 'NDCG@5': 0.60121, 'NDCG@10': 0.62264, 'NDCG@100': 0.65124, 'NDCG@1000': 0.66547}
{'MAP@1': 0.48917, 'MAP@3': 0.54537, 'MAP@5': 0.56537, 'MAP@10': 0.5756, 'MAP@100': 0.58282, 'MAP@1000': 0.58334}
{'Recall@1': 0.48917, 'Recall@3': 0.61506, 'Recall@5': 0.69156, 'Recall@10': 0.75178, 'Recall@100': 0.875, 'Recall@1000': 0.98667}
{'P@1': 0.51333, 'P@3': 0.22556, 'P@5': 0.15267, 'P@10': 0.084, 'P@100': 0.00997, 'P@1000': 0.00112}


In [22]:
# Download and load dataset
dataset = "trec-covid" # dataset name
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{dataset}.zip"
out_dir = "datasets"
data_path = util.download_and_unzip(url, out_dir)

In [23]:
# Load corpus, queries, and qrels
corpus, queries, qrels = GenericDataLoader(data_folder=data_path).load(split="test")

2025-03-19 10:53:29 - Loading Corpus...


  0%|          | 0/171332 [00:00<?, ?it/s]

2025-03-19 10:53:30 - Loaded 171332 TEST Documents.
2025-03-19 10:53:30 - Doc Example: {'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%) had comorbidities. Twenty-four isolates (60%) were associated with pneumonia, 14 (35%) with upper respiratory tract 

In [24]:
import json
# load paraphrased dataset
with open(f"{dataset}_query_paraphrased_gpt4o.json", encoding='utf-8') as f:
    # Load the JSON data into a Python dictionary
    queries_para = json.load(f)

In [25]:
queries_p = {}
for q in queries_para:
  queries_p[q] = queries_para[q]['query_p']

In [26]:
results_instructor = retriever_instructor.retrieve(corpus, queries_p)

2025-03-19 10:53:31 - Encoding Queries...


  0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:00<00:00,  6.64it/s]

2025-03-19 10:53:31 - Sorting Corpus by document length (Longest first)...


...

 99%|█████████▉| 165/167 [00:40<00:00,  6.36it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 99%|█████████▉| 166/167 [00:40<00:00,  6.59it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 167/167 [00:40<00:00,  4.09it/s]


In [29]:
# Evaluate your model with NDCG@k, MAP@K, Recall@K and Precision@K where k = [1,3,5,10,100,1000]
ndcg, _map, recall, precision = retriever_instructor.evaluate(qrels, results_instructor, retriever_instructor.k_values)

2025-03-19 12:03:33 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2025-03-19 12:03:34 - 

2025-03-19 12:03:34 - NDCG@1: 0.6000
2025-03-19 12:03:34 - NDCG@3: 0.5004
2025-03-19 12:03:34 - NDCG@5: 0.4704
2025-03-19 12:03:34 - NDCG@10: 0.4368
2025-03-19 12:03:34 - NDCG@100: 0.3284
2025-03-19 12:03:34 - NDCG@1000: 0.3398
2025-03-19 12:03:34 - 

2025-03-19 12:03:34 - MAP@1: 0.0016
2025-03-19 12:03:34 - MAP@3: 0.0034
2025-03-19 12:03:34 - MAP@5: 0.0049
2025-03-19 12:03:34 - MAP@10: 0.0085
2025-03-19 12:03:34 - MAP@100: 0.0470
2025-03-19 12:03:34 - MAP@1000: 0.1368
2025-03-19 12:03:34 - 

2025-03-19 12:03:34 - Recall@1: 0.0016
2025-03-19 12:03:34 - Recall@3: 0.0038
2025-03-19 12:03:34 - Recall@5: 0.0058
2025-03-19 12:03:34 - Recall@10: 0.0112
2025-03-19 12:03:34 - Recall@100: 0.0772
2025-03-19 12:03:34 - Recall@1000: 0.3342
2025-03-19 12:03:34 - 

2025-03-19 12:03:34 - P@1: 0.6600
2025-03-19 12:03:34

In [30]:
print(f"Model: Instructor; Dataset: {dataset} (paraphrased)")
print("-" * 150)
print(ndcg)
print(_map)
print(recall)
print(precision)

Model: Instructor; Dataset: trec-covid (paraphrased)
------------------------------------------------------------------------------------------------------------------------------------------------------
{'NDCG@1': 0.6, 'NDCG@3': 0.50039, 'NDCG@5': 0.47038, 'NDCG@10': 0.43684, 'NDCG@100': 0.3284, 'NDCG@1000': 0.33978}
{'MAP@1': 0.00158, 'MAP@3': 0.00344, 'MAP@5': 0.0049, 'MAP@10': 0.00851, 'MAP@100': 0.04703, 'MAP@1000': 0.13679}
{'Recall@1': 0.00158, 'Recall@3': 0.00378, 'Recall@5': 0.00579, 'Recall@10': 0.01123, 'Recall@100': 0.07717, 'Recall@1000': 0.33422}
{'P@1': 0.66, 'P@3': 0.53333, 'P@5': 0.492, 'P@10': 0.458, 'P@100': 0.3356, 'P@1000': 0.15974}
